In [6]:
import dask.array as da
import numpy as np 
from unyt import unyt_array

da.ones(5) - unyt_array(np.ones(5), 'm')

dask.array<sub, shape=(5,), dtype=float64, chunksize=(5,), chunktype=unyt.unyt_array>

In [7]:
(da.ones(5) - unyt_array(np.ones(5), 'm')).compute()

UnitOperationError: The <ufunc 'subtract'> operator for unyt_arrays with units "dimensionless" (dimensions "1") and "m" (dimensions "(length)") is not well defined.

https://docs.dask.org/en/latest/generated/dask.array.register_chunk_type.html



In [3]:
da.register_chunk_type(unyt_array)

In [1]:
import dask.array as da
import numpy as np 
from unyt import unyt_array

a = da.array(unyt_array(np.random.random(5), 'm'))
b = da.array(unyt_array(np.random.random(5), 'm'))

In [2]:
(a * b).compute()

unyt_array([0.01918827, 0.0236459 , 0.4259253 , 0.19213826, 0.29618294], 'm**2')

In [3]:
import time
import numpy as np 
from dask import delayed
from dask import array as dask_array
from unyt import unyt_array
# from unyt.dask_array import unyt_from_dask

# simulation of reading in data off disk into a unyt_dask array 
chunk_ids = range(30)  # this could instead be a list of files 
def get_chunk_info(field, chunk_id):
    # to build dask arrays, need to know the sizes and data types of the chunks
    # in a real application, we would need to check the size of the field we 
    # are reading in. In this case, we will set a chunk size with some random
    # variation
    chunk_size = (1000000 + chunk_id + np.random.randint(-100, 100) , 3)
    # if info on units is stored in the file, we would also want to read that in
    units = "kg/m**3"
    # same for the data type
    field_dtype = np.float64
    return chunk_size, units, field_dtype

@delayed
def read_chunk(field, chunk_id, chunk_size, field_dtype):
    # this would be where you open a chunk and read in the data, from which
    # we are constructing our delayed array. Since we are generating fake
    # data instead of reading, we also pass in chunk_size here and field_dtype
    # for consistency (if we were reading, we would not need these).    
    time.sleep(2) # add an artificial slowdown to simulate some extra process
    return unyt_array(np.random.random(chunk_size).astype(field_dtype), "kg/m**3")

chunks = []
field = "Density"
for chunk_id in chunk_ids:
    # get the size of this chunk
    chunk_size, units, field_dtype = get_chunk_info(field, chunk_id)
    # get the delayed read of this chunk
    delayed_read = read_chunk(field, chunk_id, chunk_size, field_dtype)
    # get the plain dask array of this chunk
    plain_dask = dask_array.from_delayed(delayed_read, chunk_size, dtype=field_dtype)
    chunks.append(plain_dask)

density = dask_array.concatenate(chunks)
density

dask.array<concatenate, shape=(30000298, 3), dtype=float64, chunksize=(1000124, 3), chunktype=numpy.ndarray>

In [4]:
chunks[0]

dask.array<from-value, shape=(1000048, 3), dtype=float64, chunksize=(1000048, 3), chunktype=numpy.ndarray>

In [6]:
import time
import numpy as np 
from dask import delayed
from dask import array as dask_array
from unyt import unyt_array


dask_array.register_chunk_type(unyt_array)

# simulation of reading in data off disk into a unyt_dask array 
chunk_ids = range(30)  # this could instead be a list of files 
def get_chunk_info(field, chunk_id):
    # to build dask arrays, need to know the sizes and data types of the chunks
    # in a real application, we would need to check the size of the field we 
    # are reading in. In this case, we will set a chunk size with some random
    # variation
    chunk_size = (1000 + chunk_id + np.random.randint(-100, 100) , 3)
    # if info on units is stored in the file, we would also want to read that in
    units = "kg/m**3"
    # same for the data type
    field_dtype = np.float64
    return chunk_size, units, field_dtype

@delayed
def read_chunk(field, chunk_id, chunk_size, field_dtype):
    # this would be where you open a chunk and read in the data, from which
    # we are constructing our delayed array. Since we are generating fake
    # data instead of reading, we also pass in chunk_size here and field_dtype
    # for consistency (if we were reading, we would not need these).    
    time.sleep(2) # add an artificial slowdown to simulate some extra process
    return unyt_array(np.random.random(chunk_size).astype(field_dtype), "kg/m**3")

chunks = []
field = "Density"
for chunk_id in chunk_ids:
    # get the size of this chunk
    chunk_size, units, field_dtype = get_chunk_info(field, chunk_id)
    # get the delayed read of this chunk
    delayed_read = read_chunk(field, chunk_id, chunk_size, field_dtype)
    # get the plain dask array of this chunk
    plain_dask = dask_array.from_delayed(delayed_read, chunk_size, dtype=field_dtype)
    chunks.append(plain_dask)

density = dask_array.concatenate(chunks)
density

dask.array<concatenate, shape=(30037, 3), dtype=float64, chunksize=(1120, 3), chunktype=numpy.ndarray>

In [9]:
unyt_array(density.compute(), 'm')

unyt_array([[0.41183146, 0.37838531, 0.7247127 ],
            [0.95825194, 0.99089332, 0.8065562 ],
            [0.81546628, 0.05632982, 0.0971417 ],
            ...,
            [0.83386462, 0.05324124, 0.99744009],
            [0.71139728, 0.34319763, 0.53700492],
            [0.95724118, 0.63065497, 0.34956308]], 'm')

In [10]:
from dask.distributed import client

In [12]:
c = client.Client()

In [14]:
a = c.persist(density)

In [15]:
type(a)

dask.array.core.Array

In [18]:
a.min().compute()

unyt_quantity(2.56048588e-06, 'dimensionless')